In [6]:
import re
import tweepy
import pandas as pd
import string
import nltk
from IPython.core.display_functions import display

nltk.download('stopwords')
from HanTa import HanoverTagger as ht

[nltk_data] Downloading package stopwords to /Users/galal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
api_key = 'ALR6ExjYkMBfebl33tcjXyZ4s'
api_key_secret = 'nDDTt61clONQ8isJifsLT7nqhxfjHFFlo6W8rl2SblhPMeVLk3'

access_token = '2317868736-6BMg18lkng6EIY2nrSSoISburY9QcaUovk78xPY'
access_token_secret = 'UcHtfXjvurM4dT9ZpMgvwDfZ2ukQtEGQqaLEfhueSeFyq'

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)



limit=500

keywords = '#HaltDieFresse'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('#HaltDieFresse_tweets.csv')



keywords = '#heute'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('#heute_tweets.csv')



keywords = '#DankeMerkel'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('#DankeMerkel_tweets.csv')



keywords = 'Fresse'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)


columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('Fresse_tweets.csv')



keywords = 'heute'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('heute_tweets.csv')



keywords = 'danke Merkel'

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

columns = ['Tweet']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('danke-Merkel_tweets.csv')




Unauthorized: 401 Unauthorized
89 - Invalid or expired token.

In [8]:
#vorverarbeitung von  txt
def removeEmojis(txt):
     regrex_pattern = re.compile(pattern = "["
         u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
     return regrex_pattern.sub(r'',txt)

def removeMentions(txt):
    return re.sub(r'@[A-Za-z0-9]+', '' ,txt)

def removeHashtags(txt):
    return re.sub(r'#','',txt)

def removeRetweet(txt):
    return re.sub(r'RT[\s]+','',txt)

def removeHyperlinks(txt):
    return re.sub(r'https?:\/\/\S+','',txt)

def removePunctuations(txt):
    txt  = "".join([char for char in txt if char not in string.punctuation])
    txt = re.sub('[0-9]+', '', txt)
    return txt

def txtVerarbeitung(txt):
    txt = removeEmojis(txt)
    txt = removeMentions(txt)
    txt = removeHashtags(txt)
    txt = removeRetweet(txt)
    txt = removeHyperlinks(txt)
    txt = removePunctuations(txt)
    return txt

def tokenization(txt):
    txt = re.split('\W+', txt)
    return txt

stopword = nltk.corpus.stopwords.words('german')
def removeStopwords(txt):
    txt = [word for word in txt if word not in stopword]
    return txt

hannover = ht.HanoverTagger('morphmodel_ger.pgz')
def lemmatizer(txt):
    lemmatized = []
    for word in txt:
        if word and word.strip():
            result = hannover.analyze(word)[0]
            lemmatized.append(result)
    return lemmatized

In [11]:
def generateCleanedOutput(csvFileName, outputFileName):
    df = pd.DataFrame(pd.read_csv(csvFileName), columns=['Tweet'])
    df['Tweet'] = df['Tweet'].apply(txtVerarbeitung)
    df['Tweet_tokenized'] = df['Tweet'].apply(lambda x: tokenization(x.lower()))
    df['Tweet_nonstop'] = df['Tweet_tokenized'].apply(removeStopwords)
    df['Tweet_lemmatized'] = df['Tweet_nonstop'].apply(lemmatizer)
    print(csvFileName)
    display(df)
    df['Tweet_lemmatized'].apply(lambda s: ' '.join([str(elem) for elem in s])).to_csv(outputFileName)

# Heute Tweets
generateCleanedOutput('heute_tweets.csv', 'heute_tweets_Vorverarbeitung.csv')
generateCleanedOutput('#heute_tweets.csv', '#heute_hashtag_Vorverarbeitung.csv')


# Danke Merkel Tweets
generateCleanedOutput('danke-Merkel_tweets.csv', 'danke-Merkel_tweets_Vorverarbeitung.csv')
generateCleanedOutput('#DankeMerkel_tweets.csv', '#DankeMerkel_hashtag_Vorverarbeitung.csv')

# Fresse Tweets
generateCleanedOutput('Fresse_tweets.csv', 'Fresse_tweets_Vorverarbeitung.csv')
generateCleanedOutput('#HaltDieFresse_tweets.csv', '#Fresse_hashtag_Vorverarbeitung.csv')

heute_tweets.csv


,Tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,Mal eine Frage\nIst das heute die Ruhe vor od...,"[, mal, eine, frage, ist, das, heute, die, ruh...","[, mal, frage, heute, ruhe, sturm]","[mal, Frage, heute, Ruhe, Sturm]"
1,unterwegs Alles gut blockchainopfer war ich...,"[unterwegs, alles, gut, blockchainopfer, war, ...","[unterwegs, gut, blockchainopfer, schon, wurde...","[unterwegs, gut, Blockchainopfer, schon, werde..."
2,Heute sind die Grundrechte Tage in Haft Es d...,"[, heute, sind, die, grundrechte, tage, in, ha...","[, heute, grundrechte, tage, haft, darf, verge...","[heute, Grundrecht, Tag, Haft, dürfen, Vergess..."
3,Läuft heute BVB,"[läuft, heute, bvb]","[läuft, heute, bvb]","[laufen, heute, Bvb]"
4,Also hat Bock heute\n\n\n BVBBMG,"[also, hat, bock, heute, bvbbmg]","[bock, heute, bvbbmg]","[Bock, heute, Bvbbmg]"
...,...,...,...,...
495,Mia Daniels und die praktische Führerscheinp...,"[, mia, daniels, und, die, praktische, führers...","[, mia, daniels, praktische, führerscheinprüfu...","[Mia, Daniel, praktisch, Führerscheinprüfung, ..."
496,iMTV Guuude \nheute ist der letzte Warmup Qual...,"[imtv, guuude, heute, ist, der, letzte, warmup...","[imtv, guuude, heute, letzte, warmup, qualifie...","[Imtv, Guuude, heute, letzter, Warmup, Qualifi..."
497,th Servicetweet DiscoursMaschine Corona Querde...,"[th, servicetweet, discoursmaschine, corona, q...","[th, servicetweet, discoursmaschine, corona, q...","[th, Servicetweet, Discoursmaschine, Corona, Q..."
498,Mir gehts heute nicht gut Entfolgt an einem an...,"[mir, gehts, heute, nicht, gut, entfolgt, an, ...","[gehts, heute, gut, entfolgt, tag]","[gehts, heute, gut, entfolgen, Tag]"


#heute_tweets.csv


,Tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,Teterower See\n\naktuell heute see schilf tete...,"[teterower, see, aktuell, heute, see, schilf, ...","[teterower, see, aktuell, heute, see, schilf, ...","[Teterower, See, aktuell, heute, See, Schilf, ..."
1,Auto Ankauf heute Sofort,"[auto, ankauf, heute, sofort, ]","[auto, ankauf, heute, sofort, ]","[Auto, Ankauf, heute, sofort]"
2,ZDFChinaReporter UlfRöller um bei heute kriti...,"[zdfchinareporter, ulfröller, um, bei, heute, ...","[zdfchinareporter, ulfröller, heute, kritisch,...","[Zdfchinareporter, Ulfröller, heute, kritisch,..."
3,TKS Gedenkkundgebung in Potsdam an den rechten...,"[tks, gedenkkundgebung, in, potsdam, an, den, ...","[tks, gedenkkundgebung, potsdam, rechten, terr...","[Tks, Gedenkkundgebung, Potsdam, Recht, Terror..."
4,Auto Ankauf heute Sofort Bargeld,"[auto, ankauf, heute, sofort, bargeld]","[auto, ankauf, heute, sofort, bargeld]","[Auto, Ankauf, heute, sofort, Bargeld]"
...,...,...,...,...
349,In my opinion SPSCHWEIZ DEFINES the term RETAR...,"[in, my, opinion, spschweiz, defines, the, ter...","[my, opinion, spschweiz, defines, the, term, r...","[my, Opinion, Spschweiz, Defines, The, Term, R..."
350,q qanon stasi Easterling SPY\n\nWeAreNATO worl...,"[q, qanon, stasi, easterling, spy, wearenato, ...","[q, qanon, stasi, easterling, spy, wearenato, ...","[q, Qanon, Stasi, Easterling, Spy, Wearenato, ..."
351,Heute Mensch und ich haben bis Uhr geschlafen...,"[heute, mensch, und, ich, haben, bis, uhr, ges...","[heute, mensch, uhr, geschlafen, couch, überna...","[heute, Mensch, Uhr, schlafen, Couch, übernach..."
352,Kuenstler Heute wird weltweit der PlumpuddingT...,"[kuenstler, heute, wird, weltweit, der, plumpu...","[kuenstler, heute, weltweit, plumpuddingtag, z...","[Kuenstler, heute, weltweit, Plumpuddingtag, z..."


danke-Merkel_tweets.csv


...

,Tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,ungeimpft unge simon tanzverbot haider dankem...,"[, ungeimpft, unge, simon, tanzverbot, haider,...","[, ungeimpft, unge, simon, tanzverbot, haider,...","[Ungeimpft, ung, Simon, Tanzverbot, Haider, Da..."
1,Letzter Tag mit Merkel Als sie ins Amt trat w...,"[, letzter, tag, mit, merkel, als, sie, ins, a...","[, letzter, tag, merkel, amt, trat, jahre, alt...","[letzter, Tag, Merkel, Amt, treten, Jahr, alt,..."
2,Endlich mal ein Statement das die Mehrheit d...,"[, endlich, mal, ein, statement, das, die, meh...","[, endlich, mal, statement, mehrheit, bürgerin...","[endlich, mal, Statement, Mehrheit, Bürgerinne..."
3,FriedrichMerz Erst das dreggische Haus verlass...,"[friedrichmerz, erst, das, dreggische, haus, v...","[friedrichmerz, erst, dreggische, haus, verlas...","[Friedrichmerz, erst, dreggisch, Haus, verlass..."
4,Die Vorstellungen der Regierenden und ihrer Un...,"[die, vorstellungen, der, regierenden, und, ih...","[vorstellungen, regierenden, unterstützer, hal...","[Vorstellung, regierend, Unterstützer, Halt, a..."
...,...,...,...,...
65,Tja auch dafür DankeMerkel,"[, tja, auch, dafür, dankemerkel]","[, tja, dafür, dankemerkel]","[tja, dafür, Dankemerkel]"
66,Hm wahrscheinlich gibt es eine CDU vor und ei...,"[, hm, wahrscheinlich, gibt, es, eine, cdu, vo...","[, hm, wahrscheinlich, gibt, cdu, ära, merkel,...","[hm, wahrscheinlich, geben, Cdu, Ära, Merkel, ..."
67,Haben grad ein rechtliches Problem mit Xanny …...,"[haben, grad, ein, rechtliches, problem, mit, ...","[grad, rechtliches, problem, xanny, plan, rele...","[Grad, rechtlich, Problem, Xanny, Plan, Releas..."
68,PR Impfungen und jede Currywurst und Bier se...,"[, pr, impfungen, und, jede, currywurst, und, ...","[, pr, impfungen, currywurst, bier, selber, be...","[pr, Impfung, Currywurst, Bier, selber, bezahl..."


Fresse_tweets.csv


,Tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,Was in Kanada passiert ist weltweit geplant ...,"[, was, in, kanada, passiert, ist, weltweit, g...","[, kanada, passiert, weltweit, geplant, beobac...","[Kanada, passieren, weltweit, planen, beobacht..."
1,Lauterbach Halt endlich mal die Fresse,"[lauterbach, halt, endlich, mal, die, fresse]","[lauterbach, halt, endlich, mal, fresse]","[Lauterbach, Halt, endlich, mal, fressen]"
2,Dehnungsstreifen sind voll schön und Haare am...,"[dehnungsstreifen, sind, voll, schön, und, haa...","[dehnungsstreifen, voll, schön, haare, körper,...","[Dehnungsstreifen, voll, schön, Haar, Körper, ..."
3,Meine Fresse geht das einfach BVBBMG,"[meine, fresse, geht, das, einfach, bvbbmg]","[fresse, geht, einfach, bvbbmg]","[fressen, gehen, einfach, Bvbbmg]"
4,JAAAAA MAN VERDAMMTE SCHEIßE\n\nMEINE FRESSE E...,"[jaaaaa, man, verdammte, scheiße, meine, fress...","[jaaaaa, verdammte, scheiße, fresse, ey, bvbbmg]","[Jaaaaa, verdammen, Scheiße, fressen, ey, Bvbbmg]"
...,...,...,...,...
495,bitte halt die fresse,"[, bitte, halt, die, fresse]","[, bitte, halt, fresse]","[bitte, Halt, fressen]"
496,Soooooo geht es zig Familien Wir wollen ja sc...,"[, soooooo, geht, es, zig, familien, wir, woll...","[, soooooo, geht, zig, familien, ja, schon, ga...","[Soooooo, gehen, Zig, Familie, ja, schon, gar,..."
497,DAS GESUNDHEITSSYSTEM WAR NIE ÜBERLASTET\n\nJ...,"[, das, gesundheitssystem, war, nie, überlaste...","[, gesundheitssystem, nie, überlastet, ja, maß...","[Gesundheitssystem, nie, überlasten, ja, Maßna..."
498,Wu Wenn du für Hackepeter im Kopf und das Rech...,"[wu, wenn, du, für, hackepeter, im, kopf, und,...","[wu, hackepeter, kopf, recht, ansteckung, demo...","[Wu, Hackepeter, Kopf, Recht, Ansteckung, demo..."


#HaltDieFresse_tweets.csv


,Tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,markus wenn man keine Ahnung hat heisst man Ha...,"[markus, wenn, man, keine, ahnung, hat, heisst...","[markus, ahnung, heisst, haintz, u, verbreitet...","[Markus, Ahnung, heißen, Haintz, u, verbreitet..."
1,HaltDieFresse,"[, haltdiefresse]","[, haltdiefresse]",[Haltdiefresse]
2,wiebe haltdiefresse Russentroll,"[, wiebe, haltdiefresse, russentroll]","[, wiebe, haltdiefresse, russentroll]","[Wiebe, Haltdiefresse, russentroll]"
3,Lauterbach haltdiefresse,"[, lauterbach, haltdiefresse]","[, lauterbach, haltdiefresse]","[Lauterbach, Haltdiefresse]"
4,ratzinger HaltDieFresse\n\nQuerfurzer Querden...,"[ratzinger, haltdiefresse, querfurzer, querden...","[ratzinger, haltdiefresse, querfurzer, querden...","[Ratzinger, Haltdiefresse, Querfurzer, Querden..."
...,...,...,...,...
74,Gott bist Du ein Trottel \nHaltDieFresse Cov...,"[, gott, bist, du, ein, trottel, haltdiefresse...","[, gott, trottel, haltdiefresse, covidiot, fck...","[Gott, Trottel, Haltdiefresse, Covidiot, Fcknzs]"
75,HaltDieFresse noAfD,"[haltdiefresse, noafd, ]","[haltdiefresse, noafd, ]","[Haltdiefresse, Noafd]"
76,für pseudojüdische Rassisten völkische zioNaZ...,"[, für, pseudojüdische, rassisten, völkische, ...","[, pseudojüdische, rassisten, völkische, ziona...","[pseudojüdisch, Rassisten, völkisch, Zionazis,..."
77,Siemering Habt ihr sonst noch Argumente außer...,"[, siemering, habt, ihr, sonst, noch, argument...","[, siemering, habt, argumente, außer, haltdief...","[Siemering, haben, Argument, außer, Haltdiefre..."
